In [0]:
from google.colab import files, auth, drive

In [0]:
# Mount to drive
drive.mount('/content/gdrive', force_remount=True)
data_dir_drive ='/content/gdrive/My Drive/Colab Notebooks/NLP1/Lab6'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
! pip install nltk

In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt
      Unzipping tokenizers/punkt.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

# **Knowledge-rich WSD based on WordNet++**
In this laboratory we present the technique developped by Simone Paolo Ponzetto and Roberto Navigli in their article "[Knowledge-rich Word Sense Disambiguation Rivaling Supervised Systems](https://www.aclweb.org/anthology/P10-1154.pdf)". This approach uses supplimentary relations between words, in order to compute relatedness between concepts. All the new relations are based on wikipedia, this is the reason why in this laboratory we need to use the [wikipedia module](https://wikipedia.readthedocs.io/en/latest/) (documentation: [https://wikipedia.readthedocs.io/en/latest/code.html](https://wikipedia.readthedocs.io/en/latest/code.html)). However, some of the neede relations are not implemented in the wikipedia module, therefore, we will also need to use the requests module in order to use [MediaWiki action API](https://www.mediawiki.org/wiki/API:Main_page) to wich we'll need to transmit requests.

Types of relations

"Redirect to" relations ([https://www.mediawiki.org/wiki/API:Redirects](https://www.mediawiki.org/wiki/API:Redirects))
disambiguation pages
internal links
In order to use these relations we need a **mapping** between WordNet word senses and wikipedia articles. In the article, they give as an example the word "soda" ([https://en.wikipedia.org/wiki/Soda](https://en.wikipedia.org/wiki/Soda)). Notice that the disambiguation page redirects to this same page: [https://en.wikipedia.org/wiki/Soda_(disambiguation)](https://en.wikipedia.org/wiki/Soda). You can see that it has multiple senses illustrated in a list of pages. you can obtain the ids of those pages with a code similar to :



In [0]:

import requests

def getWikiSensesByWordTitle(word):
    #create a connection(session)
    r_session = requests.Session()

    #url for the MediaWiki action API
    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query", #we are creating a query
        "titles": word, #for the title input param    
        "prop": "redirects", #asking for all the redirects (to the title car)
        "format": "json" #and we want the output in a json format
    }

    #we obtain the response to the get request with the given parmeters
    query_response = r_session.get(url=URL, params=PARAMS)
    json_data = query_response.json()

    #print(json_data)
    wikipedia_pages = json_data["query"]["pages"]

    #we iterate through items and print all the redirects (their title and id)
    try:
        L = []
        for k, v in wikipedia_pages.items():
            for redir in v["redirects"]:
                L.append((redir["title"], redir["pageid"]))
                #print("{} redirect to {}({})".format(redir["title"], v["title"], redir["pageid"]))
        return L
    except KeyError as err:
        if err.args[0]=='redirects':
            #print("It has no redirects")
            return []
        else:
            print(repr(err))


In [0]:
def getWikiSensesByWordId(id):
    #create a connection(session)
    r_session = requests.Session()

    #url for the MediaWiki action API
    URL = "https://en.wikipedia.org/w/api.php"

    PARAMS = {
        "action": "query", #we are creating a query
        "pageids": id, #for the id input param    
        "prop": "redirects", #asking for all the redirects (to the title car)
        "format": "json" #and we want the output in a json format
    }

    #we obtain the response to the get request with the given parmeters
    query_response = r_session.get(url=URL, params=PARAMS)
    json_data = query_response.json()

    #print(json_data)
    wikipedia_pages = json_data["query"]["pages"]

    #we iterate through items and print all the redirects (their title and id)
    try:
        L = []
        for k, v in wikipedia_pages.items():
            for redir in v["redirects"]:
                L.append((redir["title"], redir["pageid"]))
                #print("{} redirect to {}({})".format(redir["title"], v["title"], redir["pageid"]))
        return L
    except KeyError as err:
        if err.args[0]=='redirects':
            #print("It has no redirects")
            return []
        else:
            print(repr(err))


Notice the normalization field, it is not what you might expect; it doesn't obtain the lemma, or apply any transformation on the letter case, it is about [Unicode normalization](http://www.unicode.org/reports/tr15/).

for disambiaguations, notice the following two links:

[https://en.wikipedia.org/w/api.php?action=query&titles=soda&prop=pageprops&format=json](https://en.wikipedia.org/w/api.php?action=query&titles=soda&prop=pageprops&format=json)
[https://en.wikipedia.org/w/api.php?action=query&titles=car&prop=pageprops&format=json](https://en.wikipedia.org/w/api.php?action=query&titles=car&prop=pageprops&format=json)
In order to create the mapping we shall use for a given wikipedia page:

sense labels (actually they are the titles of the pages. At the time when the article was written, the titles had this syntax "word(sense label)" like "soda(soft drink)", however, notice that now you only find the sense label as a title.
links (outgoing links from the current page)
categories
The article uses the notation Ctx(w) for the set of words obtained from the text of some or all these pages.

Next, we need the WordNet context for a sense s, Ctx(s), for each sense of the word. For this we take the following relations:

synonymy
hypernymy/hyponymy
sisterhood (senses that have the same direct hypernym)
gloss
The next step is the mapping

For each word that we want to disambiguate, if we have **only one sense**, and only one wikipedia page, we map taht wikipedia page to the word.
In the case of multiple senses, for each remaining wikipedia word w(after the mapping from the former step) that still has no associate Wordnet word, we take all the redirects to the word w. For each such redirect we look if we already have a mapping associated to it (a relation between its sense and the wikipedia page). If we have such a mapping and the mapped word is in w's sysnset, we map w to the sense associated to the redirect page
For all wikipedia pages that aren't mapped yet, we try to assign the most probable sense. The most probable sense has the highest value p computed as score(w,sense)/sum, where sum is the sum between all the combinations of scores between each sense of the word from wordnet and each senese of the word from wikipedia. the score is the number of common words between the context of the sense in wikipedia and the context of the sense in WordNet to which we add 1 score(s,w)=|Ctx(s) ∩ Ctx(w)|+1
In the end we have created new relations (WordNet++) that we can use in a **simplified Lesk manner** to disambiguate a text. We will compute the overlaps on all the glosses given by the mentioned relations.

In [0]:
word = 'soda'

In [0]:
from nltk.corpus import wordnet

WNsenses = wordnet.synsets(word)
print(WNsenses)

[Synset('sodium_carbonate.n.01'), Synset('pop.n.02')]


In [0]:
wikiSenses = getWikiSensesByWordTitle(word)
print(wikiSenses)

[('SODA', 3318069), ('Sodas', 5599353), ('Soday', 13752636), ('Soda (disambiguation)', 19677220), ('Soda drink', 47486780)]


In [0]:
result = {}

In [0]:
#lines 1-2
for w in wikiSenses:
    result[w] = None

result

{('SODA', 3318069): None,
 ('Soda (disambiguation)', 19677220): None,
 ('Soda drink', 47486780): None,
 ('Sodas', 5599353): None,
 ('Soday', 13752636): None}

In [0]:
#lines 3-5
for w in wikiSenses:
    wiki = getWikiSensesByWordId(w[1])
    wn = wordnet.synsets(w[0])
    print(wiki, wn)
    if (len(wiki) == 1 or len(wiki) == 0) and len(wn) == 1:
        result[w] = wn[0]

result

[] [Synset('sodium_carbonate.n.01'), Synset('pop.n.02')]
[] [Synset('sodium_carbonate.n.01'), Synset('pop.n.02')]
[] []
[] []
[] []


{('SODA', 3318069): None,
 ('Soda (disambiguation)', 19677220): None,
 ('Soda drink', 47486780): None,
 ('Sodas', 5599353): None,
 ('Soday', 13752636): None}

In [0]:
#lines 6-10
for w in wikiSenses:
    if result[w] is None:
        wiki = getWikiSensesByWordId(w[1])
        for d in wiki:
            if result[d] is not None and result[w] in wordnet.synsets(w[0]):
                result[w] = result[d]
                break

result

{('SODA', 3318069): None,
 ('Soda (disambiguation)', 19677220): None,
 ('Soda drink', 47486780): None,
 ('Sodas', 5599353): None,
 ('Soday', 13752636): None}

In [0]:
! pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=1b24ea7c5c2acff6b622dd6b4346ab87aed48a86190587f3e1dccb3a9b24eb9b
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import wikipedia

def getContextWikipedia(title, id):
    wikipedia.set_lang("en")
    try:
        page = wikipedia.page(title = title, pageid = id)
        links = page.links
        categs = page.categories
    except:
        return title

    link_s = ''
    for link in links:
        link_s += link + ' '
    categ_s = ''
    for categ in categs:
        categ_s += categ + ' '

    return title + ' ' + link_s + categ_s


In [0]:
def getSisterhood(syn):
    L = []

    for hypernym in syn.hypernyms():
        hyponyms = hypernym.hyponyms()
        L += hyponyms
    return L

In [0]:
def getContextWordNet(syn):
    similar_to_s = ''
    for e in syn.similar_tos():
        similar_to_s += e.name()[:e.name().find('.')] + ' '

    hypernym_s = ''
    for hypernym in syn.hypernyms():
        hypernym_s += hypernym.name()[:hypernym.name().find('.')] + ' '

    meronym_s = ''
    meronyms = syn.substance_meronyms() + syn.part_meronyms() + syn.part_meronyms()
    for meronym in meronyms:
        meronym_s += meronym.name()[:meronym.name().find('.')] + ' '

    sister_s = ''
    for sister in getSisterhood(syn):
        sister_s += sister.name()[:sister.name().find('.')] + ' '

    return syn.definition() + ' ' + similar_to_s + hypernym_s + meronym_s + sister_s

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from nltk.corpus import wordnet

def compare(gloss1, gloss2):
    words1 = word_tokenize(gloss1)
    words2 = word_tokenize(gloss2)

    words1 = [x.lower() for x in words1 if x.isalnum()]
    words2 = [x.lower() for x in words2 if x.isalnum()]

    #print(gloss1, gloss2)
    #print(words1, words2)

    ps=nltk.PorterStemmer()

    stem1 = list(map(lambda x: ps.stem(x), words1))
    stem2 = list(map(lambda x: ps.stem(x), words2))

    #print(stem1, stem2)

    r = 1;
    for s1 in stem1:
        for s2 in stem2:
            if s1 == s2:
                r += 1

    return r

In [0]:
def intersect(title, id, syn):

    wiki = getContextWikipedia(title, id)
    wn = getContextWordNet(syn)

    return compare(wiki, wn)

    return None

In [0]:
#lines 11-14
maxSum = 0
for w in wikiSenses:
        for syn in wordnet.synsets(word):
            i = intersect(w[0], w[1], syn)
            maxSum += i
        
for w in wikiSenses:
    if result[w] is None:
        max = 0
        s = None
        for syn in wordnet.synsets(word):
            i = intersect(w[0], w[1], syn) / maxSum
            #print(w,s,i)
            if i > max:
                max = i
                s = syn
        result[w] = s

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [0]:
result

{('SODA', 3318069): Synset('sodium_carbonate.n.01'),
 ('Soda (disambiguation)', 19677220): Synset('sodium_carbonate.n.01'),
 ('Soda drink', 47486780): Synset('sodium_carbonate.n.01'),
 ('Sodas', 5599353): Synset('sodium_carbonate.n.01'),
 ('Soday', 13752636): Synset('sodium_carbonate.n.01')}

In [0]:
with open(data_dir_drive + '/out.txt', 'a') as the_file:
    the_file.write(str(result))